In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl
from collections import Counter
import math

# courses = load_json('data/courses.txt')
stopwords = load_pkl('data/MovieSummaries/stopwords.pkl')

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import string

nltk.download('omw')

[nltk_data] Downloading package omw to /Users/azizlaadhar/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

In [7]:
#imports
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import json
import Helper
####from transformers import BertTokenizer, BertModel
###import torch


# Remove punctuation
symbols = "!\"#$%&()*-+./:;<=>?@[\]^_`{|}~\n,\'\’\‘\“\”\—"

In [8]:
import re

def remove_text_in_braces(input_text):
    """Remove text between curly braces {} in a given string."""
    pattern = re.compile(r'\{.*?\}')
    cleaned_text = re.sub(pattern, '', input_text)
    return cleaned_text


In [26]:
# Pre-processing function
def preprocess_text(data):
    print (data)
    # Convert bytes-like objects to string 
    data = str(data)
    #data = remove_text_in_braces(data)
    # Tokenize
    data = word_tokenize(data)

    # Lowercase
    data = np.char.lower(data)

    for i in symbols:
        data = np.char.replace(data, i, '')

    # Remove apostrophe
    np.char.replace(data, "'", "")
 
    # Remove stopwords
    data = [word for word in data if word not in stopwords]

    # Remove single character words
    data = [word for word in data if len(word) > 1]

    # Stemming
    stemmer = PorterStemmer()
    
    data = [stemmer.stem(word) for word in data]

    print (data)
    return data

These preprocessing methods serve different purposes and aim to improve the quality and usefulness of the text data. Here's an explanation of each method and why it was chosen:


- Tokenization: Tokenization is the process of splitting text into individual tokens or words. It helps break down the text into smaller units, making it easier to process and analyze. The word_tokenize() function from NLTK is used to perform tokenization in the code.

- Removal of Separation Words: Separation words like "courseId," "name," and "description" may not provide meaningful information for analysis. Removing them helps eliminate noise and focus on the important content of the text.

- Removal of Stopwords and Punctuation: Stopwords are common words that do not carry significant meaning in a text, such as "the," "is," or "and." Removing stopwords helps reduce the noise in the text data and improves the accuracy of analysis. Punctuation marks are also removed as they often don't contribute to the semantics of the text.

- Stemming: Stemming reduces words to their base or root form by removing suffixes. It helps in normalizing the text data, reducing the vocabulary size, and capturing the essence of words. The Porter stemming algorithm is applied in the code using the PorterStemmer from NLTK.


In [24]:
# # Read the "courses.txt" file
# with open('data/courses.txt', 'r') as file:
#     lines = file.readlines()
events_df = pd.read_csv('data/MovieSummaries/events.csv')

events_df['Tokenized Description'] = events_df['Event Description'].apply(preprocess_text) 

# Pre-process the corpus
# plot_summaries_df['vectorized_summary'] = plot_summaries_df['plot_summary'].apply(preprocess_text)

The Compromise of 1850 was a package of five separate bills passed by the United States Congress, which defused a four-year political confrontation between slave and free states on the status of territories acquired in the Mexican–American War. It aimed at maintaining balance and quelling tensions between the North and South. The Compromise ended the slave trade (but not slavery) in Washington, D.C., and admitted California as a free state.
['compromis', '1850', 'packag', 'separ', 'bill', 'pass', 'unit', 'state', 'congress', 'defus', 'fouryear', 'polit', 'confront', 'slave', 'free', 'state', 'statu', 'territori', 'acquir', 'mexican–american', 'war', 'aim', 'maintain', 'balanc', 'quell', 'tension', 'north', 'south', 'compromis', 'end', 'slave', 'trade', 'slaveri', 'washington', 'dc', 'admit', 'california', 'free', 'state']
The Australian Gold Rush began with the discovery of gold nuggets in New South Wales, Australia. The event sparked a massive influx of immigrants to the country, part

In [27]:
plot_summaries_df = pd.read_csv('data/MovieSummaries/plot_summaries.txt', sep='\t', header=None, 
                names=['wiki_movie_id', 
                        'plot_summary'])

plot_summaries_df['plot_summary_tok'] = plot_summaries_df['plot_summary'].apply(preprocess_text) 

In [167]:
def doc_freq(DF, word):
    # Returns the frequency of a word in all documents
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [168]:
corpus = preprocessed_corpus
# Compute document frequencies for each term
DF = {}
for i in range(len(corpus)):
    tokens = corpus[i]
    for w in tokens:
        if w in DF:
            DF[w].add(i)
        else:
            DF[w] = {i}
total_vocab = [x for x in DF]
for i in DF:
    DF[i] = len(DF[i])

# Compute TF-IDF scores
n = len(corpus)
doc = 0

tf_idf = {}

for i in range(n):
    
    tokens = corpus[i]
    
    counter = Counter(tokens )
    words_count = len(tokens )
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(DF, token)
        idf = np.log((n+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1


In [172]:
# Print the 15 terms in the description of the com308 class with the highest TF-IDF scores






IndexError: invalid index to scalar variable.

If a term has a high frequency within a document (high TF), and it occurs in relatively fewer documents across the collection (low IDF), its TF-IDF score will be high. This indicates that the term is both important within the document and relatively unique to that document.

On the other hand, if a term has a low frequency within a document (low TF), or it occurs in many documents across the collection (high IDF), its TF-IDF score will be low. This suggests that the term is less significant within the document or it is commonly found across multiple documents.

## Exercise 4.3: Document similarity search

In [122]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [123]:
D = np.zeros((n, len(total_vocab)))

for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [124]:
def gen_vector(tokens):
    
    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(DF,token)
        idf = math.log((n+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [158]:
def cosine_similarity(k, query):

    # Preprocess the query the same way we did with the documents
    tokens = preprocess_text(query)

    print("\nQuery:", query)
    print("")
    print(tokens)
    
    # Generate query vector
    query_vector = gen_vector(tokens)
    
    # Calculate cosine similarity between query vector and all document vectors
    d_cosines = []
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))


    # Sort the cosine similarities
    d_cosines = np.array(d_cosines).argsort()

    #top k indexes
    top_indexes = d_cosines[-k:][::-1]

    # Return the top k matching documents
    return top_indexes


In [160]:
query1 = "markov chains"
top1 = cosine_similarity(5, query1)
print("\nTop 5 documents for the query \"markov chains\":")
for i in top1:
    print(f"Document {i}: {courses[i]['name']}")


query2 = "facebook"
top2 = cosine_similarity(5, query2) 
print("\nTop 5 documents for the query \"facebook\":")
for i in top2:      
    print(f"Document {i}: {courses[i]['name']}")





Query: markov chains

['markov', 'chain']

Top 5 documents for the query "markov chains":
Document 398: Applied probability & stochastic processes
Document 80: Applied stochastic processes
Document 245: Markov chains and algorithmic applications
Document 44: Supply chain management
Document 99: Mathematical models in supply chain management

Query: facebook

['facebook']

Top 5 documents for the query "facebook":
Document 798: Computational Social Media
Document 291: Hydrogeophysics
Document 289: Electronic properties of solids and superconductivity
Document 288: CCMX Advanced Course - Instrumented Nanoindentation
Document 287: Molecular and cellular biophysic II


In [7]:
!pip install torch --upgrade


In [8]:
!pip install stanfordnlp --upgrade

In [17]:
import stanfordnlp

def encode_text_with_corenlp(text, server_url='http://localhost:9000'):
    nlp = stanfordnlp.Pipeline(server_url=server_url)
    doc = nlp(text)
    return doc

# Example text
example_text = "Stanford CoreNLP is a suite of natural language processing tools."

# Encode text
encoded_doc = encode_text_with_corenlp(example_text)

# Accessing token information
for sentence in encoded_doc.sentences:
    for token in sentence.tokens:
        print(f"Word: {token.text}, POS: {token.pos}, NER: {token.ner}")


OSError: dlopen(/Users/azizlaadhar/opt/anaconda3/envs/ada/lib/python3.9/site-packages/torch/lib/libtorch_global_deps.dylib, 0x000A): Library not loaded: @loader_path/../../functorch/.dylibs/libiomp5.dylib
  Referenced from: /Users/azizlaadhar/opt/anaconda3/envs/ada/lib/python3.9/site-packages/torch/lib/libtorch_global_deps.dylib
  Reason: tried: '/Users/azizlaadhar/opt/anaconda3/envs/ada/lib/python3.9/site-packages/torch/lib/../../functorch/.dylibs/libiomp5.dylib' (no such file), '/usr/local/lib/libiomp5.dylib' (no such file), '/usr/lib/libiomp5.dylib' (no such file)